# Complaint Data Scraped 

In [1]:
import pandas as pd
import numpy as np
import scipy as sp 
import matplotlib as plt
import matplotlib.pyplot as pltp
import plotly.plotly as py 
import missingno as msno
from statsmodels.graphics.mosaicplot import mosaic
import seaborn as sns
from collections import Counter
from __future__ import print_function
%matplotlib inline
pd.set_option('max_rows',300)
pd.set_option('max_columns',200)

In [2]:
xl = pd.ExcelFile('../Data/Complaint_Data_Scraped.xlsx')
sheetnames = xl.sheet_names  # see all sheet names
sheetnames

[u'Panel complaint data scraped',
 u'EIB complaint data scraped',
 u'MICI complaint data scraped',
 u'CAO complaint data scraped',
 u'ADB complaint data scraped']

In [3]:
dfs = {}
for name in sheetnames:
    df = xl.parse(name)
    dfs[name.split()[0]] = df
    print(name.split()[0],df.shape)
    print(df.columns,'\n','*'*40)


Panel (122, 31)
Index([u'IAM', u'Year', u'Country', u'Project', u'ID', u'IAM ID', u'Filer(s)',
       u'Environmental Category', u'Project Company', u'Project Number',
       u'Related Project Number', u'Project Type', u'Financial Institution',
       u'Project Loan Amount', u'Sector', u'Issues', u'Complaint Status',
       u'Filing Date', u'Registration Start Date', u'Registration End Date',
       u'Eligibility Start Date', u'Eligibility End Date',
       u'Dispute Resolution Start Date', u'Dispute Resolution End Date',
       u'Compliance Review Start Date', u'Compliance Review End Date',
       u'Monitoring Start Date', u'Monitoring End Date', u'Date Closed',
       u'Documents', u'Hyperlink'],
      dtype='object') 
 ****************************************
EIB (197, 31)
Index([u'IAM', u'Year', u'Country', u'Project', u'ID', u'IAM ID', u'Filer(s)',
       u'Environmental Category', u'Project Company', u'Project Number',
       u'Related Project Number', u'Project Type', u'Financia

In [6]:
dfs['EIB']

,IAM,Year,Country,Project,ID,IAM ID,Filer(s),Environmental Category,Project Company,Project Number,Related Project Number,Project Type,Financial Institution,Project Loan Amount,Sector,Issues,Complaint Status,Filing Date,Registration Start Date,Registration End Date,Eligibility Start Date,Eligibility End Date,Dispute Resolution Start Date,Dispute Resolution End Date,Compliance Review Start Date,Compliance Review End Date,Monitoring Start Date,Monitoring End Date,Date Closed,Documents,Hyperlink
0,EIB,2018,Spain,Spain Gas Network Expansion II,SG/E/2018/18,29,Confidential,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Assessment,16/02/2018,2018-02-16,22/02/2018,22/02/2018,Ongoing,NaT,NaT,NaN,NaN,NaT,NaN,NaT,NaN,http://www.eib.org/about/accountability/compla...
1,EIB,2018,Spain,Spain Gas Network Expansion II,SG/E/2018/17,29,Confidential,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Assessment,14/02/2018,2018-02-14,22/02/2018,22/02/2018,Ongoing,NaT,NaT,NaN,NaN,NaT,NaN,NaT,NaN,http://www.eib.org/about/accountability/compla...
2,EIB,2018,Italy,Trans Adriatic Pipeline,SG/E/2018/19,29,Confidential,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Assessment,09/02/2018,2018-09-02,22/02/2018,22/02/2018,Ongoing,NaT,NaT,NaN,NaN,NaT,NaN,NaT,NaN,http://www.eib.org/about/accountability/compla...
3,EIB,2018,Spain,Spain Gas Network Expansion II,SG/E/2018/15,29,Confidential,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Assessment,06/02/2018,2018-06-02,22/02/2018,22/02/2018,Ongoing,NaT,NaT,NaN,NaN,NaT,NaN,NaT,NaN,http://www.eib.org/about/accountability/compla...
4,EIB,2018,Italy,Trans Adriatic Pipeline,SG/E/2018/11,29,Confidential,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Assessment,06/02/2018,2018-06-02,2018-08-02 00:00:00,2018-08-02 00:00:00,Ongoing,NaT,NaT,NaN,NaN,NaT,NaN,NaT,NaN,http://www.eib.org/about/accountability/compla...
5,EIB,2018,Spain,Spain Gas Network Expansion II,SG/E/2018/14,29,Confidential,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Assessment,05/02/2018,2018-05-02,2018-08-02 00:00:00,2018-08-02 00:00:00,Ongoing,NaT,NaT,NaN,NaN,NaT,NaN,NaT,NaN,http://www.eib.org/about/accountability/compla...
6,EIB,2018,Italy,Trans Adriatic Pipeline,SG/E/2018/10,29,Confidential,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Assessment,05/02/2018,2018-05-02,2018-08-02 00:00:00,2018-08-02 00:00:00,Ongoing,NaT,NaT,NaN,NaN,NaT,NaN,NaT,NaN,http://www.eib.org/about/accountability/compla...
7,EIB,2018,Greece,Trans Adriatic Pipeline,SG/E/2018/12,29,Confidential,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Assessment,04/02/2018,2018-04-02,2018-08-02 00:00:00,2018-08-02 00:00:00,Ongoing,NaT,NaT,NaN,NaN,NaT,NaN,NaT,NaN,http://www.eib.org/about/accountability/compla...
8,EIB,2018,Spain,Spain Gas Network Expansion II,SG/E/2018/13,29,Confidential,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Assessment,02/02/2018,2018-02-02,2018-08-02 00:00:00,2018-08-02 00:00:00,Ongoing,NaT,NaT,NaN,NaN,NaT,NaN,NaT,NaN,http://www.eib.org/about/accountability/compla...
9,EIB,2018,Italy,Trans Adriatic Pipeline,SG/E/2018/09,29,Confidential,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Assessment,01/02/2018,2018-01-02,2018-08-02 00:00:00,2018-08-02 00:00:00,Ongoing,NaT,NaT,NaN,NaN,NaT,NaN,NaT,NaN,http://www.eib.org/about/accountability/compla...


** Inital Thoughts**

Why don't they have the same columns, ADB only has 7, MICI has a 'Hyperlink' and 'Hyperlink [Correct]', need to check that this is unaltered output from the scrapers. 

Other thoughts for doing comparison
1. Do we want them to be aggregated into a single file for comparison?  or just compare to the output files per scraper ? 
2. At the DataDive should we have the teams just work with one scrapers output, and then a fake set of altered data ... this might make more sense. 
3. Need to figure out how to create fake changed data. 

**Check On Shared Total **

In [59]:
def shared_columns(df1,df2):
    col1 = set(df1.columns.tolist())
    col2 = set(df2.columns.tolist())
    
    print( "Shared Cols: ", col1 & col2)
    return list(col1 & col2)

def add_some_prefix(df,prefix,dupe_cols):
    df.columns = [prefix +c if c not in dupe_cols else c for c in df.columns]
    return df


def shared_summary(df_list, print_per_col=True):
    file_cols = []
    for df in df_list:
        cols = df_list[df].columns.tolist()
        for c in cols:
            file_cols.append((df,c))
    shared = pd.DataFrame(file_cols,columns=['colname','table'])
    shared = shared.groupby('table').agg(lambda x: list(x))
    shared['cnt'] = shared.colname.apply(len)
#     shared = shared[shared.cnt > 1]
    shared = shared.sort_values('cnt',ascending=False)
    if print_per_col:
        for i in shared.index:
            print ( 'Col: ',i,'||', len(shared))
            for v in shared.loc[i].values[0]:
                print (' '*10 + '--> ' + v)
    return shared.reset_index()


In [60]:
shared = shared_summary(dfs, print_per_col=False)

In [61]:
shared

,table,colname,cnt
0,Project Number,"[ADB, EIB, CAO, MICI, Panel]",5
1,Country,"[ADB, EIB, CAO, MICI, Panel]",5
2,IAM,"[ADB, EIB, CAO, MICI, Panel]",5
3,Sector,"[ADB, EIB, CAO, MICI, Panel]",5
4,Related Project Number,"[EIB, CAO, MICI, Panel]",4
5,Registration Start Date,"[EIB, CAO, MICI, Panel]",4
6,Registration End Date,"[EIB, CAO, MICI, Panel]",4
7,Project Type,"[EIB, CAO, MICI, Panel]",4
8,Compliance Review End Date,"[EIB, CAO, MICI, Panel]",4
9,Project Loan Amount,"[EIB, CAO, MICI, Panel]",4
